In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
# pd.options.display.float_format = '{:,.2f}'.format


import matplotlib 
from matplotlib import pyplot as plt
from matplotlib import style
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [5]:
df_oneStream= pd.read_excel(r'C:\Users\A803377\Downloads\Reporting_CubeData_TableauTieTo_M5.xlsx',sheet_name=1)
df_oneStream

,Cube,Entity,Parent,Cons,Scenario,Time,View,Account,Flow,Origin,IC,UD1,UD2,UD3,UD4,UD5,UD6,UD7,UD8,Amount,HasData,Annotation,Assumptions,AuditComment,Footnote,VarianceExplanation
0,Reporting,1020,NaN,USD,Actual_FPA,2023M5,Periodic,Revenue,TOP,Top,Top,NA_PP,900,TOP,TOP,16,None,Top,None,-2.131344e+05,Yes,NaN,NaN,NaN,NaN,NaN
1,Reporting,1020,NaN,USD,Actual_FPA,2023M5,Periodic,Revenue,TOP,Top,Top,NA_PP,922,TOP,TOP,1N,None,Top,None,6.454860e+03,Yes,NaN,NaN,NaN,NaN,NaN
2,Reporting,1020,NaN,USD,Actual_FPA,2023M5,Periodic,Revenue,TOP,Top,Top,NA_PP,920,TOP,TOP,16,None,Top,None,5.541000e+01,Yes,NaN,NaN,NaN,NaN,NaN
3,Reporting,1020,NaN,USD,Actual_FPA,2023M5,Periodic,Revenue,TOP,Top,Top,NA_PP,901,TOP,TOP,1P,None,Top,None,6.562006e+05,Yes,NaN,NaN,NaN,NaN,NaN
4,Reporting,1020,NaN,USD,Actual_FPA,2023M5,Periodic,Revenue,TOP,Top,Top,NA_PP,903,TOP,TOP,1R,None,Top,None,1.159414e+06,Yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7857,Reporting,9641,NaN,NZD,Actual_FPA,2023M5,Periodic,MGMT_GR_Prof,TOP,Top,Top,Intercompany,906,TOP,TOP,99,None,Top,None,-2.840037e+04,Yes,NaN,NaN,NaN,NaN,NaN
7858,Reporting,9641,NaN,NZD,Actual_FPA,2023M5,Periodic,MGMT_GR_Prof,TOP,Top,Top,Intercompany,910,TOP,TOP,99,None,Top,None,-2.422300e+03,Yes,NaN,NaN,NaN,NaN,NaN
7859,Reporting,9641,NaN,NZD,Actual_FPA,2023M5,Periodic,MGMT_GR_Prof,TOP,Top,Top,Intercompany,911,TOP,TOP,99,None,Top,None,-1.066117e+04,Yes,NaN,NaN,NaN,NaN,NaN
7860,Reporting,9641,NaN,NZD,Actual_FPA,2023M5,Periodic,MGMT_GR_Prof,TOP,Top,Top,Intercompany,914,TOP,TOP,99,None,Top,None,-4.735260e+03,Yes,NaN,NaN,NaN,NaN,NaN


# Fiscal Period 

In [6]:
Fiscal_Period = df_oneStream.loc[0," Time"]
Fiscal_Period= Fiscal_Period.replace('M', '00')
Fiscal_Period

'2023005'

In [7]:
df_oneStream = df_oneStream[[' Entity',' Account',' UD1',' Amount']]
df_oneStream.rename(columns = {' Entity':'Company Code',' UD1': 'Reporting_Line',' Account':'Account',' Amount':'Amount'}, inplace = True)
df_oneStream= df_oneStream.groupby(["Company Code","Account","Reporting_Line"],as_index=False)["Amount"].sum()
df_oneStream = df_oneStream.pivot_table(index=["Company Code","Reporting_Line"],columns='Account',values='Amount',aggfunc='sum',).fillna(0)
df_oneStream.rename(columns = {'MGMT_GR_Prof':'GP'}, inplace = True)
df_oneStream.reset_index(inplace=True)

# Changes
df_oneStream['Company Code'] = df_oneStream['Company Code'].apply(lambda x: '{0:0>4}'.format(x))
df_oneStream["Reporting_Line"]= df_oneStream["Reporting_Line"].replace({'LAM_IWT':'LAM_IS', 'LAM_PP':'LAM_CS','APAC_PP':'AP_CS','APAC_IWT':'AP_IS',
                                                                    'EMEA_PP':'EMEA_CS','EMEA_IWT':'EMEA_IS','NA_PP':'NA_CS','NA_IWT':'NA_IS',
                                                                   'PS_EMEA':'EMEA_PS','PS_LA':'SA_PS','PS_NAM':'NA_PS','PS_SA':'ZA_PS'},regex=True)

df_oneStream["Reporting_Line"]= df_oneStream["Reporting_Line"].replace('None', 'NVOther',regex=True)
df_oneStream["Reporting_Line"]= df_oneStream["Reporting_Line"].replace('No_SG', 'NVOther',regex=True)
df_oneStream = df_oneStream.groupby(['Company Code','Reporting_Line'],as_index=False).sum()

df_oneStream

Account,Company Code,Reporting_Line,GP,Revenue
0,0070,EMEA_CS,1.544140e+08,3.060225e+08
1,0070,EMEA_IS,4.516748e+07,1.695405e+08
2,0070,Intercompany,-3.105098e+07,0.000000e+00
3,0070,NVOther,-3.472557e+04,0.000000e+00
4,0072,EMEA_IS,4.723226e+05,0.000000e+00
...,...,...,...,...
208,ZA01,EMEA_CS,5.047823e+04,0.000000e+00
209,ZA01,EMEA_IS,7.841974e+06,2.030903e+07
210,ZA01,Intercompany,-2.858348e+05,7.559840e+03
211,ZA01,NVOther,-2.447970e+03,0.000000e+00


In [8]:
df_oneStream['Fiscal Period'] = Fiscal_Period

In [9]:
print('GP:',df_oneStream['GP'].sum())
print('NR:',df_oneStream['Revenue'].sum())

GP: 14373841345.29034
NR: 62696036244.62399


In [10]:
df_oneStream['Reporting_Line'].sort_values().drop_duplicates()

106           AP_CS
108           AP_IS
208         EMEA_CS
54          EMEA_IS
27          EMEA_PS
157    Intercompany
162          LAM_CS
68           LAM_IS
142           NA_CS
164           NA_IS
147           NA_PS
56          NVOther
65              RSA
154           SA_PS
212           ZA_PS
Name: Reporting_Line, dtype: object

In [11]:
df_dest= pd.read_excel(r'C:\Users\A803377\Downloads\Destination 2023005.xlsx',sheet_name=0)
df_dest['CompanyCode'] = df_dest['CompanyCode'].apply(lambda x: '{0:0>4}'.format(x))
df_dest["Reporting_Line"]= df_dest["Reporting_Line"].replace(np.nan, 'NVOther',regex=True)
df_dest["Reporting_Line"]= df_dest["Reporting_Line"].replace('Specialties', 'NVOther',regex=True)
df_dest = df_dest.drop(df_dest[df_dest['CompanyCode'] == '0000'].index)
df_dest["Fiscal Period"] = df_dest["Fiscal Period"].astype('string')
df_dest[['$LC Rev','$LC GP']] = df_dest[['$LC Rev','$LC GP']]*1000
df_dest.rename(columns = {'CompanyCode':'Company Code'}, inplace = True)
df_dest

,Company Code,Reporting_Line,Fiscal Period,$LC Rev,$LC GP
0,FR07,EMEA_PS,2023005,4.600473e+06,969664.280
1,5500,LAM_IS,2023005,2.464846e+07,7954512.101
2,0612,AP_IS,2023005,2.693604e+06,903371.789
3,5500,SA_PS,2023005,0.000000e+00,-71593.601
4,9641,AP_IS,2023005,3.118342e+05,110799.664
...,...,...,...,...,...
451,3585,EMEA_CS,2023005,0.000000e+00,0.000
452,0927,EMEA_CS,2023005,0.000000e+00,0.000
453,MX01,LAM_CS,2023005,0.000000e+00,0.000
454,0464,EMEA_IS,2023005,0.000000e+00,0.000


In [12]:
df_dest['Reporting_Line'].sort_values().drop_duplicates()
print('GP:',df_dest['$LC GP'].sum())
print('NR:',df_dest['$LC Rev'].sum())

GP: 14574921822.457502
NR: 62682609382.910095


In [14]:
# Merging Mapping And Difference File
df_new = pd.merge(df_oneStream,df_dest,on=['Company Code','Reporting_Line','Fiscal Period'],how='outer')

df_new[['GP','Revenue','$LC Rev','$LC GP']]= df_new[['GP','Revenue','$LC Rev','$LC GP']].replace(np.nan, 0.0,regex=True)
df_new['$LC Rev Diff'] =  df_new['$LC Rev'] - df_new['Revenue']
df_new['$LC GP Diff'] =  df_new['$LC GP']- df_new['GP']

df_new

,Company Code,Reporting_Line,GP,Revenue,Fiscal Period,$LC Rev,$LC GP,$LC Rev Diff,$LC GP Diff
0,0070,EMEA_CS,1.544140e+08,3.060225e+08,2023005,3.060225e+08,1.544718e+08,0.000000e+00,5.782657e+04
1,0070,EMEA_IS,4.516748e+07,1.695405e+08,2023005,1.695405e+08,4.508422e+07,-2.980232e-08,-8.325922e+04
2,0070,Intercompany,-3.105098e+07,0.000000e+00,2023005,0.000000e+00,-3.105098e+07,0.000000e+00,-3.725290e-09
3,0070,NVOther,-3.472557e+04,0.000000e+00,2023005,0.000000e+00,-3.472557e+04,0.000000e+00,0.000000e+00
4,0072,EMEA_IS,4.723226e+05,0.000000e+00,2023005,0.000000e+00,8.051506e+04,0.000000e+00,-3.918076e+05
...,...,...,...,...,...,...,...,...,...
449,3585,EMEA_CS,0.000000e+00,0.000000e+00,2023005,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
450,0927,EMEA_CS,0.000000e+00,0.000000e+00,2023005,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
451,MX01,LAM_CS,0.000000e+00,0.000000e+00,2023005,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
452,0464,EMEA_IS,0.000000e+00,0.000000e+00,2023005,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [145]:
df_Mapping= pd.read_excel(r'C:\Users\A803377\Downloads\Topsides Mapping.xlsx',sheet_name=0)
df_Mapping

,Reporting_Line,Sales Group,Customer Group
0,AP_CS,SG_PAP,16
1,AP_IS,SG_WAP,17
2,EMEA_CS,SG_PEU,16
3,EMEA_IS,SG_WEU,17
4,EMEA_PS,SG_PLK,PG
5,Intercompany,SG_999,99
6,LAM_CS,SG_PLA,16
7,LAM_IS,SG_WLA,17
8,NA_CS,SG_PNA,16
9,NA_IS,SG_WNA,17


In [146]:
print('GP Difference:',df_new['$LC GP'].sum()- df_oneStream['GP'].sum())
print('NR Difference:',df_new['$LC Rev'].sum() - df_oneStream['Revenue'].sum())

GP Difference: 198016515.49499512
NR Difference: -10410336.130691528


In [147]:
print('GP:',df_new['$LC GP Diff'].sum())
print('NR:',df_new['$LC Rev Diff'].sum())

GP: 198016515.4949962
NR: -10410336.130667448


In [148]:
# Merging Mapping And Difference File

df_final = pd.merge(df_new,df_Mapping,on=['Reporting_Line'],how='outer')
df_final

,Company Code,Reporting_Line,GP,Revenue,Fiscal Period_x,Fiscal Period_y,$LC Rev,$LC GP,$LC Rev Diff,$LC GP Diff,Sales Group,Customer Group
0,0070,EMEA_CS,"149,565,280.82","312,592,283.32",2023003,2023003,"312,592,283.32","149,578,733.46",0.00,"13,452.63",SG_PEU,16
1,0271,EMEA_CS,"5,472,372.11","23,527,034.35",2023003,2023003,"23,527,034.35","6,109,404.96",0.00,"637,032.85",SG_PEU,16
2,0310,EMEA_CS,"329,975.59","1,390,486.39",2023003,2023003,"1,390,486.39","329,975.56",0.00,-0.03,SG_PEU,16
3,0322,EMEA_CS,"197,088.04","1,307,361.90",2023003,2023003,"1,307,361.90","268,814.67",0.00,"71,726.63",SG_PEU,16
4,0330,EMEA_CS,"1,534,867.99","5,109,136.64",2023003,2023003,"5,109,136.64","1,536,592.13",0.00,"1,724.14",SG_PEU,16
...,...,...,...,...,...,...,...,...,...,...,...,...
436,1026,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17
437,1999,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17
438,1000,NA_IS,0.00,0.00,NaN,2023003,0.00,-0.02,0.00,-0.02,SG_WNA,17
439,0923,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17


In [149]:
print('GP:',df_final['$LC GP Diff'].sum())
print('NR:',df_final['$LC Rev Diff'].sum())

GP: 198016515.494996
NR: -10410336.13066717


In [150]:
# '$LC GP Diff':'$LC COGS GP',
# Changing Naming Convention 
df_final.rename(columns = {'Revenue':'$LC OS Rev','GP':'$LC OS GP',
                           '$LC Rev':'$LC EDW Rev','$LC GP':'$LC EDW GP'}, inplace = True)
df_final['$LC Final Rev Diff'] =  df_final['$LC Rev Diff'] * -1
df_final['$LC Topsides COGS'] = df_final['$LC GP Diff'] -df_final['$LC Rev Diff']
df_final['Fiscal Period'] = Fiscal_Period
df_final['Datasource'] = 'Controllers Topsides'

df_final

,Company Code,Reporting_Line,$LC OS GP,$LC OS Rev,Fiscal Period_x,Fiscal Period_y,$LC EDW Rev,$LC EDW GP,$LC Rev Diff,$LC GP Diff,Sales Group,Customer Group,$LC Final Rev Diff,$LC Topsides COGS,Fiscal Period,DataSource
0,0070,EMEA_CS,"149,565,280.82","312,592,283.32",2023003,2023003,"312,592,283.32","149,578,733.46",0.00,"13,452.63",SG_PEU,16,-0.00,"13,452.63",2023003,Controllers Topsides
1,0271,EMEA_CS,"5,472,372.11","23,527,034.35",2023003,2023003,"23,527,034.35","6,109,404.96",0.00,"637,032.85",SG_PEU,16,-0.00,"637,032.85",2023003,Controllers Topsides
2,0310,EMEA_CS,"329,975.59","1,390,486.39",2023003,2023003,"1,390,486.39","329,975.56",0.00,-0.03,SG_PEU,16,-0.00,-0.03,2023003,Controllers Topsides
3,0322,EMEA_CS,"197,088.04","1,307,361.90",2023003,2023003,"1,307,361.90","268,814.67",0.00,"71,726.63",SG_PEU,16,-0.00,"71,726.63",2023003,Controllers Topsides
4,0330,EMEA_CS,"1,534,867.99","5,109,136.64",2023003,2023003,"5,109,136.64","1,536,592.13",0.00,"1,724.14",SG_PEU,16,-0.00,"1,724.14",2023003,Controllers Topsides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,1026,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides
437,1999,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides
438,1000,NA_IS,0.00,0.00,NaN,2023003,0.00,-0.02,0.00,-0.02,SG_WNA,17,-0.00,-0.02,2023003,Controllers Topsides
439,0923,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides


In [151]:
# df_final['$LC Topsides COGS'] = df_final['$LC GP Diff'] -df_final['$LC Rev Diff']

print('NR:',df_final['$LC Final Rev Diff'].sum())
print('COGS:',df_final['$LC Topsides COGS'].sum())

NR: 10410336.13066717
COGS: 208426851.62566385


In [152]:
df_final[['$LC Topsides COGS','$LC GP Diff','$LC Final Rev Diff']]= df_final[['$LC Topsides COGS','$LC GP Diff','$LC Final Rev Diff']].replace(np.nan, 0.0,regex=True)
df_final.to_excel('TopSides $LC 2023004 - v1.xlsx',index=False)

file_name_Top = 'TopSides $LC' + Fiscal_Period + '.xlsx'

df_final.to_excel(file_name_Top,index=False)
df_final

,Company Code,Reporting_Line,$LC OS GP,$LC OS Rev,Fiscal Period_x,Fiscal Period_y,$LC EDW Rev,$LC EDW GP,$LC Rev Diff,$LC GP Diff,Sales Group,Customer Group,$LC Final Rev Diff,$LC Topsides COGS,Fiscal Period,DataSource
0,0070,EMEA_CS,"149,565,280.82","312,592,283.32",2023003,2023003,"312,592,283.32","149,578,733.46",0.00,"13,452.63",SG_PEU,16,-0.00,"13,452.63",2023003,Controllers Topsides
1,0271,EMEA_CS,"5,472,372.11","23,527,034.35",2023003,2023003,"23,527,034.35","6,109,404.96",0.00,"637,032.85",SG_PEU,16,-0.00,"637,032.85",2023003,Controllers Topsides
2,0310,EMEA_CS,"329,975.59","1,390,486.39",2023003,2023003,"1,390,486.39","329,975.56",0.00,-0.03,SG_PEU,16,-0.00,-0.03,2023003,Controllers Topsides
3,0322,EMEA_CS,"197,088.04","1,307,361.90",2023003,2023003,"1,307,361.90","268,814.67",0.00,"71,726.63",SG_PEU,16,-0.00,"71,726.63",2023003,Controllers Topsides
4,0330,EMEA_CS,"1,534,867.99","5,109,136.64",2023003,2023003,"5,109,136.64","1,536,592.13",0.00,"1,724.14",SG_PEU,16,-0.00,"1,724.14",2023003,Controllers Topsides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,1026,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides
437,1999,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides
438,1000,NA_IS,0.00,0.00,NaN,2023003,0.00,-0.02,0.00,-0.02,SG_WNA,17,-0.00,-0.02,2023003,Controllers Topsides
439,0923,NA_IS,0.00,0.00,NaN,2023003,0.00,0.00,0.00,0.00,SG_WNA,17,-0.00,0.00,2023003,Controllers Topsides


In [153]:
df_final2= df_final[["Company Code","Reporting_Line","Customer Group","Sales Group",'$LC Final Rev Diff','$LC Topsides COGS','Fiscal Period','DataSource']]
df_final3 = df_final2.loc[~((df_final2['$LC Final Rev Diff'] == 0.0) & (df_final2['$LC Topsides COGS'] == 0.0))]
df_final3 = df_final2.loc[~((df_final2['$LC Final Rev Diff'] == -0.0) & (df_final2['$LC Topsides COGS'] == -0.0))]
df_final3 = df_final2.loc[~((df_final2['$LC Final Rev Diff'] == -0.0) & (df_final2['$LC Topsides COGS'] == 0.0))]
df_final3 = df_final2.loc[~((df_final2['$LC Final Rev Diff'] == 0.0) & (df_final2['$LC Topsides COGS'] == -0.0))]

df_final3

file_name_Top_Final = 'TopSides Final $LC' + Fiscal_Period + '.xls'

df_final3.to_excel(file_name_Top_Final,index=False)

In [154]:
df_final3['Sales Group'].sort_values().drop_duplicates()

376     No_SG
113    SG_999
287    SG_PAP
0      SG_PEU
212    SG_PL0
251    SG_PLA
275    SG_PLE
205    SG_PLK
239    SG_PNA
440    SG_PZK
234    SG_RSA
314    SG_WAP
63     SG_WEU
267    SG_WLA
438    SG_WNA
Name: Sales Group, dtype: object

In [155]:
df_final3

,Company Code,Reporting_Line,Customer Group,Sales Group,$LC Final Rev Diff,$LC Topsides COGS,Fiscal Period,DataSource
0,0070,EMEA_CS,16,SG_PEU,-0.00,"13,452.63",2023003,Controllers Topsides
1,0271,EMEA_CS,16,SG_PEU,-0.00,"637,032.85",2023003,Controllers Topsides
2,0310,EMEA_CS,16,SG_PEU,-0.00,-0.03,2023003,Controllers Topsides
3,0322,EMEA_CS,16,SG_PEU,-0.00,"71,726.63",2023003,Controllers Topsides
4,0330,EMEA_CS,16,SG_PEU,-0.00,"1,724.14",2023003,Controllers Topsides
...,...,...,...,...,...,...,...,...
432,CA02,NA_IS,17,SG_WNA,-0.00,"-5,958.08",2023003,Controllers Topsides
433,US33,NA_IS,17,SG_WNA,"-67,021.69","-512,854.50",2023003,Controllers Topsides
434,US40,NA_IS,17,SG_WNA,-0.00,"-25,000.00",2023003,Controllers Topsides
438,1000,NA_IS,17,SG_WNA,-0.00,-0.02,2023003,Controllers Topsides
